In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [13:01<00:00, 18.74it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
119778,Robbery,76,2021-03-31,Juice WRLD,https://open.spotify.com/track/6Hj9jySrnFppAI0...,United States,top200,MOVE_UP,310581.000,"08 Dec 2019, 19:12","22 May 2018, 14:48","Jarad Anthony Higgins (born December 2, 1998; ...","Jarad Anthony Higgins (born December 2, 1998; ...",Hip-Hop,emo rap,rap,cloud rap,trap,Hip-Hop,sad,emo rap,Juice WRLD,emo,spotify:track:6iaSML1PIYq936g62BDtBq,0.623,0.692,2.000,-5.122,1.000,0.050,0.328,0.000,0.153,0.566,79.899,audio_features,6iaSML1PIYq936g62BDtBq,https://api.spotify.com/v1/tracks/6iaSML1PIYq9...,https://api.spotify.com/v1/audio-analysis/6iaS...,240051.000,4.000,2,1030,hip-hop,hip-hop
111435,Sunflower - Spider-Man: Into the Spider-Verse,67,2021-10-04,"Post Malone, Swae Lee",https://open.spotify.com/track/0RiRZpuVRbi7oqR...,United States,top200,MOVE_DOWN,298359.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Post+Malon...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:3KkXRkHbMCARz0aVfEt68P,0.760,0.479,2.000,-5.574,1.000,0.047,0.556,0.000,0.070,0.913,89.911,audio_features,3KkXRkHbMCARz0aVfEt68P,https://api.spotify.com/v1/tracks/3KkXRkHbMCAR...,https://api.spotify.com/v1/audio-analysis/3KkX...,158040.000,4.000,5,834,NaN,other
7254,In the Name of Love,47,2017-01-27,"Martin Garrix, Bebe Rexha",https://open.spotify.com/track/04DwTuZ2VBdJCCC...,United States,top200,MOVE_DOWN,415803.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Martin+Gar...",,netherlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:23L5CiUhw2jV1OIMwthR3S,0.501,0.519,4.000,-5.880,0.000,0.041,0.109,0.000,0.454,0.168,133.990,audio_features,23L5CiUhw2jV1OIMwthR3S,https://api.spotify.com/v1/tracks/23L5CiUhw2jV...,https://api.spotify.com/v1/audio-analysis/23L5...,195707.000,4.000,18,150,netherlands,other
353796,Tell Em,71,2021-12-28,"Cochise, $NOT",https://open.spotify.com/track/7nc7mlSdWYeFom8...,United States,top200,MOVE_DOWN,288998.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Cochise,+$...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:7nc7mlSdWYeFom84zZ8Wr8,0.672,0.717,5.000,-7.476,1.000,0.226,0.103,0.000,0.398,0.473,157.905,audio_features,7nc7mlSdWYeFom84zZ8Wr8,https://api.spotify.com/v1/tracks/7nc7mlSdWYeF...,https://api.spotify.com/v1/audio-analysis/7nc7...,180380.000,4.000,17,231,NaN,other
289304,Good Thing (with Kehlani),47,2019-10-13,Zedd,https://open.spotify.com/track/5aUSEPNd3m5xliF...,United States,top200,SAME_POSITION,378707.000,NaN,"16 May 2011, 05:54",\nZedd is a Grammy-award winning electro-house...,\nZedd is a Grammy-award winning electro-house...,electronic,electro house,House,electro,dubstep,NaN,NaN,NaN,NaN,NaN,spotify:track:5aUSEPNd3m5xliFK4pn5mU,0.693,0.564,1.000,-3.857,0.000,0.038,0.399,0.000,0.041,0.574,185.913,audio_features,5aUSEPNd3m5xliFK4pn5mU,https://api.spotify.com/v1/tracks/5aUSEPNd3m5x...,https://api.spotify.com/v1/audio-analysis/5aUS...,210901.000,3.000,7,84,electronic,electronic


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.434
female       0.146
group        0.086
non-binary   0.017
Name: gender, dtype: float64

In [34]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
112215,Boss Bitch,197,2020-02-13,Doja Cat,https://open.spotify.com/track/78qd8dvwea0Gosb...,United States,top200,NEW_ENTRY,224891.000,"17 Jun 2021, 21:38","12 Sep 2016, 19:40","Amala Zandile Dlamini, known professionally as...","Amala Zandile Dlamini, known professionally as...",pop,rnb,rap,Hip-Hop,female vocalists,pop,rap,pop rap,Soundtrack,electronic,spotify:track:78qd8dvwea0Gosb6Fe6j3k,0.707,0.955,10.000,-4.593,0.000,0.222,0.127,0.000,0.202,0.575,125.989,audio_features,78qd8dvwea0Gosb6Fe6j3k,https://api.spotify.com/v1/tracks/78qd8dvwea0G...,https://api.spotify.com/v1/audio-analysis/78qd...,134240.000,4.000,17,174,pop,pop,female,NaN
242382,Walk,169,2018-12-03,Comethazine,https://open.spotify.com/track/21kOVEG3bDCVphK...,United States,top200,MOVE_UP,268407.000,"04 Aug 2022, 03:16","28 Aug 2018, 21:25","Frankie Jahmier Childress (b. on July 6th, 199...","Frankie Jahmier Childress (b. on July 6th, 199...",Hip-Hop,rap,trap,cloud rap,hip hop,NaN,NaN,NaN,NaN,NaN,spotify:track:0sVcnEo2jnUnr3p7uVDZ9o,0.828,0.795,1.000,-6.188,1.000,0.260,0.228,0.000,0.307,0.739,160.099,audio_features,0sVcnEo2jnUnr3p7uVDZ9o,https://api.spotify.com/v1/tracks/0sVcnEo2jnUn...,https://api.spotify.com/v1/audio-analysis/0sVc...,96064.000,4.000,26,174,hip-hop,hip-hop,male,NaN
445322,The Way You Used To Do,7,2017-06-22,Queens of the Stone Age,https://open.spotify.com/track/7dBMQbBKG6n9HkV...,United States,viral50,MOVE_DOWN,NaN,"03 Feb 2019, 08:36","10 Feb 2006, 12:27",\nQueens of the Stone Age is a rock band from ...,\nQueens of the Stone Age is a rock band from ...,Stoner Rock,rock,alternative rock,seen live,hard rock,Stoner Rock,rock,2017,alternative rock,hard rock,spotify:track:52LL3IFB8N3PaJmoZ8Xii1,0.482,0.843,0.000,-5.909,1.000,0.288,0.010,0.103,0.362,0.433,97.907,audio_features,52LL3IFB8N3PaJmoZ8Xii1,https://api.spotify.com/v1/tracks/52LL3IFB8N3P...,https://api.spotify.com/v1/audio-analysis/52LL...,274282.000,4.000,4,7,stoner rock,indie rock,male,NaN
126797,Whiskey Glasses,131,2021-10-23,Morgan Wallen,https://open.spotify.com/track/6foY66mWZN0pSRj...,United States,top200,MOVE_UP,240856.000,"06 Dec 2021, 23:10","14 Jan 2018, 02:26",Morgan Wallen is a country music singer and so...,Morgan Wallen is a country music singer and so...,country,racist,male vocalist,the voice,racism,NaN,NaN,NaN,NaN,NaN,spotify:track:6foY66mWZN0pSRjZ408c00,0.614,0.680,6.000,-4.580,1.000,0.029,0.369,0.000,0.115,0.707,149.959,audio_features,6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347.000,4.000,53,971,country,country,NaN,"May 13, 1993"
261825,Walk Me Home,197,2019-03-23,P!nk,https://open.spotify.com/track/7DHApOHsIFo7Eyf...,United States,top200,MOVE_DOWN,210620.000,"23 Feb 2019, 03:45","11 Feb 2006, 20:02","Alecia Beth Moore (born September 8, 1979 in D...","Alecia Beth Moore (born September 8, 1979 in D...",pop,pop rock,female vocalists,rock,female,pop,folk,pop rock,2010s,2019,spotify:track:7DHApOHsIFo7EyfieArYGE,0.652,0.453,1.000,-6.119,1.000,0.044,0.051,0.000,0.179,0.432,88.039,audio_features,7DHApOHsIFo7EyfieArYGE,https://api.spotify.com/v1/tracks/7DHApOHsIFo7...,https://api.spotify.com/v1/audio-analysis/7DHA...,177391.000,3.000,33,60,pop,pop,female,"September 8, 1979"


Age from artist

-- taking the data from the summary

In [29]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d*, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

December 2, 1998     10839
July 4, 1995         10700
January 23, 1998      9272
October 24, 1986      8719
December 18, 2001     7645
                     ...  
April 14, 1993           1
May 6, 2008              1
 1970, 1971              1
August 19, 1986          1
August 30, 1997          1
Name: birthday, Length: 999, dtype: int64

In [30]:
df.sample()

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
19248,White Iverson,94,2017-04-13,Post Malone,https://open.spotify.com/track/6eT7xZZlB2mwyzJ...,United States,top200,MOVE_UP,271816.000,"14 May 2022, 09:53","07 Jun 2015, 09:50","Austin Richard Post (born July 4, 1995), known...","Austin Richard Post (born July 4, 1995), known...",Hip-Hop,rap,trap,cloud rap,pop,Hip-Hop,rap,trap,2015,cloud rap,spotify:track:6eT7xZZlB2mwyzJ2sUKG6w,0.675,0.562,7.000,-7.678,1.000,0.035,0.233,0.000,0.082,0.309,130.098,audio_features,6eT7xZZlB2mwyzJ2sUKG6w,https://api.spotify.com/v1/tracks/6eT7xZZlB2mw...,https://api.spotify.com/v1/audio-analysis/6eT7...,256533.000,4.000,56,543,hip-hop,hip-hop,male,"July 4, 1995"


# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df